In [1]:
import pandas as pd
import re
import datetime
import numpy as np
import requests
from pandas.io.json import json_normalize
import json
import os
import os.path
import snowflake.connector
import boto3
from __future__ import print_function
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import random

## https://developers.google.com/sheets/api/quickstart/python
## CREDS



## In[2]:


## https://developers.google.com/sheets/api/quickstart/python

## CREDS
# S3
BUCKET = 'scale-crawler-enriched-csv-exports-us-west-2'
s3 = boto3.client('s3')
session = boto3.Session()

# Google Sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1ycZEbsg7hEb_kKAYmIg6eK0hBIl4fvhK0FDan1f5UkE'
RANGE_NAME = 'Sheet9!A:M'
PATH_TO_SECRETS_FILE = 'credentials.json'
creds = None

con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [4]:
def uploadData(data,filename):
    s3.put_object(
        ACL='bucket-owner-full-control',
        Body=data.encode('utf-8'),
        Bucket=RESULTS_BUCKET,
        Key=f'flamingo_qa_potential_issues/{filename}')

## Pull data from Google Sheet https://docs.google.com/spreadsheets/d/1UCIE1P6PbI9odzxFUjNF44s-SaPePbDUnHQKqxa9XpM/edit#gid=774020952
def pullFromGS(SCOPES,PATH_TO_SECRETS_FILE,creds,SPREADSHEET_ID,RANGE_NAME):
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(PATH_TO_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,range=RANGE_NAME).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
        
    except HttpError as err:
        print(err)
        
    df = pd.DataFrame(values[1:],columns = values[0])    
    return df

def getCQRResults(min_date,max_date):
    
    sql = f'''
    with cqr_result as (
      with audits as (
        select
          sa.CATALOG_ID,
          sa.domain,
          sa.BODY_S3_KEY,
          sa._id audit_id,
          date(sa.completed_at) audit_time,
          sa.grade :"scores" :"descriptionScore" :"score" as CQR_DESCRIPTION_SCORE,
          sa.grade :"scores" :"titleScore" :"score" as Title,
          sa.result
        from
          PUBLIC.SPOTTERAUDITS sa
          inner join (
            select
              max(completed_at) as max_time,
              CATALOG_ID
            from
              PUBLIC.SPOTTERAUDITS
            group by
              CATALOG_ID
          ) as cqr_max on cqr_max.CATALOG_ID = sa.CATALOG_ID
          and cqr_max.max_time = sa.completed_at
        where
          AUDIT_TYPE = 'Attributes'
          and sa.COMPLETED_AT is not null
          and sa.grade :"scores" :"descriptionScore" :"score" is not null
          and date(sa.completed_at) >= '{min_date}'
          and date(sa.completed_at) <= '{max_date}'
      )
      select
        au.CATALOG_ID,
        au.domain,
        au.audit_id,
        au.audit_time CQR_AUDIT_DATE,
        au.BODY_S3_KEY,
        a.key variant_id,
        b.key attribute,
        b.value :result :: string attribute_grade,
        b.value :reason :: string reason,
        b.value :comment :: string comment  
      from
        audits au,
        lateral flatten (input => au.result) a,
        lateral flatten (input => a.value) b
      where
        b.key in ('description')
        and b.value :result = 'Incorrect'
    )
    select 
    c.*,
    pv.pvid,
    pv.scraped_attributes:link::string link
    from cqr_result c
    join productvariants pv on pv.unique_id = c.variant_id
    '''
    print('Getting CQR data from Snowflake!')
    cs.execute(sql)
    df = cs.fetch_pandas_all()
    print('Success! Got CQR data from Snowflake. Number of rows:',len(df),'\n-------------')
    return df
        
def getCQRInputs(cqr_results):
    
    df = pd.DataFrame() 
    print('Getting CQR input data from S3!')
    for s3_file in cqr_results['BODY_S3_KEY'].unique().tolist():
        print('Pulling from', s3_file)
        response = s3.get_object(Bucket = BUCKET, Key = s3_file)
        tmp = pd.read_csv(response.get("Body"))
        df = pd.concat([df,tmp])
    print('Success! Got CQR input data from S3. Number of rows:', len(df),'\n-------------')
    return df

def mergeCQRData(cqr_results, cqr_inputs):
    if len(cqr_results) == 0 or len(cqr_inputs) == 0: 
        print('ERROR: Not enough information to complete')
        df = pd.DataFrame()
    else:
        print('Merging data!')
        df = cqr_results.merge(cqr_inputs[['pvid','description','link']], left_on = 'PVID', right_on = 'pvid')
        df = df.fillna('').rename(columns = {'description':'POST_PROCESSED_DESCRIPTION','COMMENT':'CORRECT_DESCRIPTION'})

        df = df.sort_values(['POST_PROCESSED_DESCRIPTION'])
        df = df.loc[(df['POST_PROCESSED_DESCRIPTION'] != '') & (df['CORRECT_DESCRIPTION'] != '')] 
        print('Success! Merged data. Number of rows:', len(df),'\n-------------')
    return df

def getPPQAData(relevant_pvids):
    pvids = "('" + "','".join(relevant_pvids) + "')"
#     print(pvids)
    sql_descs = f'''
    select
      user_email,
      _ID,
      metadata :pvids description_id,
      b.value :: string pvid,
      CREATED_AT variant_pped_at,
      metadata: auditLevel :: string audit_level,
      metadata: fieldCurrent :: string QA_DESCRIPTION
    from
      PUBLIC.QAEVENTS,
      lateral flatten(input => metadata :pvids) b
    where
      audit_level != 'Other'
      and METADATA :action in ('Save', 'SwitchItem')
      and metadata: fieldCurrent is not Null
      and pvid in {pvids}
    '''

    sql_rules = f'''
    select
      user_email,
      metadata :pvids description_id,
      b.value :: string pvid,
      CREATED_AT variant_pped_at,
      metadata: auditLevel :: string audit_level,
      metadata: flagComment :: string flagtext,
      metadata: ruleCreated :: string ruleCreated
    from
      PUBLIC.QAEVENTS,
      lateral flatten(input => metadata :pvids) b
    where
      audit_level != 'Other'
      and METADATA :action in ('CreateRule')
      and metadata: flagComment is not Null
      and pvid in {pvids}
    '''
    

    
    print('Getting descriptions data from Snowflake!')
    cs.execute(sql_descs)
    pp_desc_data = cs.fetch_pandas_all()
    print('Success! Got descriptions data from Snowflake. Number of rows:',len(pp_desc_data))    
    
    print('Getting rules data from Snowflake!')
    cs.execute(sql_rules)
    pp_rules_data = cs.fetch_pandas_all()
    print('Success! Got rules data from Snowflake. Number of rows:',len(pp_rules_data),'\n-------------')    
    
  
    
    return pp_desc_data, pp_rules_data

def generateSpeedAuditErrors(cqr_data, pp_desc_data):
    cols = ['CQR_AUDIT_DATE', 'USER_EMAIL', 'type', 'AUDIT_LEVEL',
                    'DOMAIN', 'description_PPed_at', 'sample_pvid',
                    'sample_link','CORRECT_DESCRIPTION', 'QA_DESCRIPTION',
                   'Extra text (not removed by QA)',
                   'Missing text (incorrectly removed by QA)','outcome']
        
    if len(cqr_data) == 0 or len(pp_desc_data) == 0: 
        print('ERROR: Not enough information to complete')
        dff = pd.DataFrame(columns = cols)
    else: 
        print('Generating Speed Audit errors!')
        df = cqr_data.merge(pp_desc_data, on = 'PVID')
        df = df.rename(columns = {'COMMENT':'CORRECT_DESCRIPTION'})
        df['clean_final_desc'] = df.apply(lambda x: re.sub('\\\\n|\n| ','',x['CORRECT_DESCRIPTION']),axis=1)
        df['clean_fieldcurrent'] = df.apply(lambda x: re.sub('\\\\n|\n| ','',x['QA_DESCRIPTION']),axis=1)
        df['is_correct_desc'] = df['clean_final_desc'] == df['clean_fieldcurrent']
        df = df.drop_duplicates() # .loc[df['is_correct_desc'] == False]
        if len(df) ==0:
            return df
        else:
            tmp_cols = ['CQR_AUDIT_DATE',
                'USER_EMAIL',
                'AUDIT_LEVEL',
                'DOMAIN',
                'CORRECT_DESCRIPTION',
                'QA_DESCRIPTION','is_correct_desc']

            dff = df.groupby(tmp_cols)['VARIANT_PPED_AT','PVID','LINK'].min()                .reset_index()                .rename(columns = {'VARIANT_PPED_AT':'description_PPed_at','PVID':'sample_pvid','LINK':'sample_link'})
            dff['Extra text (not removed by QA)'] = dff.apply(lambda x: np.setdiff1d([i.strip('. ').strip('! ').strip('? ').lower() for i in re.split('\. |\n|\! |\? ', x['QA_DESCRIPTION']) if i != ''],[i.strip('. ').strip('! ').strip('? ').lower() for i in re.split('\. |\n|\! |\? ', x['CORRECT_DESCRIPTION']) if i != '']), axis = 1)    
            dff['Missing text (incorrectly removed by QA)'] = dff.apply(lambda x: np.setdiff1d([i.strip('. ').strip('! ').strip('? ').lower() for i in re.split('\. |\n|\! |\? ', x['CORRECT_DESCRIPTION']) if i != ''],[i.strip('. ').strip('! ').strip('? ').lower() for i in re.split('\. |\n|\! |\? ', x['QA_DESCRIPTION']) if i != '']), axis = 1)
            dff['type'] = 'Speed Audit'
            dff['outcome'] = dff.apply(lambda x: 'incorrect speed audit' if x['is_correct_desc'] == False else 'correct speed audit', axis = 1)
           
            print('Success! Generated Speed Audit Errors\n-------------')     
        return dff.loc[:,cols]

def generateFlagAuditErrors(full_cqr_data, pp_rules_data):
    cols = ['CQR_AUDIT_DATE', 'USER_EMAIL','type','AUDIT_LEVEL',
                'DOMAIN', 'description_PPed_at', 'sample_pvid',
                'sample_link','FLAGTEXT', 'RULECREATED',
               'Extra text (not removed by QA)',
               'Missing text (incorrectly removed by QA)','outcome']
    if len(full_cqr_data) == 0 or len(pp_rules_data) == 0: 
        print('ERROR: Not enough information to complete')
        dff = pd.DataFrame(columns = cols)
    else: 
        print('Generating Flag Audit errors!')
        df = full_cqr_data.merge(pp_rules_data, on = 'PVID')
        df = df.rename(columns = {'COMMENT':'CORRECT_DESCRIPTION'})
        cols = ['CQR_AUDIT_DATE',
            'USER_EMAIL',
            'AUDIT_LEVEL',
            'DOMAIN',
            'POST_PROCESSED_DESCRIPTION',
            'CORRECT_DESCRIPTION',
               'FLAGTEXT','RULECREATED']
        dff = df.groupby(cols)['VARIANT_PPED_AT','PVID','LINK'].min()            .reset_index()            .rename(columns = {'VARIANT_PPED_AT':'description_PPed_at','PVID':'sample_pvid','LINK':'sample_link'})
        dff['Extra text (not removed by QA)'] = dff.apply(lambda x: np.setdiff1d([i.strip('. ') for i in re.split('\. |\n|\! |\? ', x['POST_PROCESSED_DESCRIPTION']) if i != ''],[i.strip('. ') for i in re.split('\. |\n|\! |\? ', x['CORRECT_DESCRIPTION']) if i != '']), axis = 1)    
        dff['Missing text (incorrectly removed by QA)'] = dff.apply(lambda x: np.setdiff1d([i.strip('. ') for i in re.split('\. |\n|\! |\? ', x['CORRECT_DESCRIPTION']) if i != ''],[i.strip('. ') for i in re.split('\. |\n|\! |\? ', x['POST_PROCESSED_DESCRIPTION']) if i != '']), axis = 1)

        dff['bad_removal'] = dff.apply(lambda x: x['RULECREATED'] == 'true' and re.sub("\.|\'|\,",'',x['FLAGTEXT'].strip().lower()) in re.sub("\.|\'|\,",'',str(x['Missing text (incorrectly removed by QA)']).strip().lower()),axis = 1)
        dff['bad_inclusion'] = dff.apply(lambda x:  x['RULECREATED'] == 'false' and re.sub("\.|\'|\,",'',x['FLAGTEXT'].strip().lower()) in re.sub("\.|\'|\,",'',str(x['Extra text (not removed by QA)']).strip().lower()),axis = 1)
        dff['outcome'] = dff.apply(lambda x: 'bad flag removal' if x['bad_removal'] == True else ('bad flag inclusion' if x['bad_inclusion'] == True else 'ok'), axis = 1)
        dff['type'] = 'Flag Audit'

        print('Success! Generated Flag Audit Errors\n-------------')    
    return dff.loc[:,cols] #dff['outcome'] != 'ok',

def completeErrorReport(speed_audit_errors,flag_audit_errors):
    df = pd.concat([speed_audit_errors,flag_audit_errors])[['CQR_AUDIT_DATE',
    'USER_EMAIL',
    'type',
    'AUDIT_LEVEL',
    'DOMAIN',
    'description_PPed_at',
    'sample_pvid',
    'sample_link',
    'CORRECT_DESCRIPTION',
    'QA_DESCRIPTION',
    'FLAGTEXT',
    'RULECREATED',
    'Extra text (not removed by QA)',
    'Missing text (incorrectly removed by QA)',
    'outcome']]
    df = df.sort_values(['DOMAIN','USER_EMAIL'])
    
    print(df.shape)
    
    df['CQR_AUDIT_DATE'] = pd.to_datetime(df['CQR_AUDIT_DATE'],utc=True)
    df['description_PPed_at'] = pd.to_datetime(df['description_PPed_at'],utc=True)
    df = df.loc[abs((df['CQR_AUDIT_DATE'] - df['description_PPed_at']).dt.days) <= 7] ## only include work done in past week
    
    df.loc[df['outcome'].isin(['incorrect speed audit','bad flag removal'])].to_clipboard(index = False)
    print('Error Report created!')
    return df
    





# In[ ]:






In [5]:
def getPPIDData(relevant_pvids):
    pvids1 = "('" + "','".join(relevant_pvids) + "')"

sql_ids =f'''
    with fb as (
  SELECT
    qe._id as ID,
    qe.user_email :: string uemail,
    qe.metadata: action action,
    qe.metadata: auditField auditField,
    qe.metadata: domain domain,
    qe.metadata: eventCreatedAt eventCreatedAt,
    qe.metadata: fieldAdditions fieldAdditions,
    qe.metadata: fieldBefore fieldBefore,
    qe.metadata: fieldCurrent fieldCurrent,
    qe.metadata: fieldLength fieldLength,
    qe.metadata: ruleCreated ruleCreated,
    qe.metadata: fieldRemovals fieldRemovals,
    qe.metadata: flagsAvailable flagsAvailable,
    qe.metadata: hintsAvailable hintsAvailable,
--  qe.metadata: pvids pvids,
    qe.metadata: totalMillisElapsed totalMillisElapsed,
    qe.event_type,
    qe.event_at,
    cast (qe.event_at as DATE) AS EventDate,
    qe.metadata: auditLevel :: string auditLevel,
    pv.value:: string as pvid
  FROM
    PUBLIC.QAEVENTS qe,
    lateral flatten(input => qe.metadata: pvids) pv
--    lateral flatten(input => pv.value, outer => true) pvd
   -- AND qe.event_type = 'submit_description_speed_audit' --  AND qe.metadata: auditLevel !='QA'
    -- AND qe.user_email LIKE '%tele%'
)
select
  fb.ID,
  fb.uemail,
  fb.action,
  fb.auditField,
  fb.domain,
  fb.eventCreatedAt,
  fb.fieldAdditions,
  fb.fieldBefore,
  fb.fieldCurrent,
  fb.fieldLength,
  fb.fieldRemovals,
  fb.ruleCreated,
  fb.flagsAvailable,
  fb.hintsAvailable,
  fb.pvid,
  fb.totalMillisElapsed,
  fb.event_type,
  fb.event_at,
  fb.EventDate
from
  fb
WHERE fb.pvid IN {pvids1}
    '''

print('Getting error IDs data from Snowflake!')
cs.execute(sql_ids)
pp_ids_data = cs.fetch_pandas_all()
print('Success! Got all error IDs data from Snowflake. Number of rows:',len(pp_ids_data),'\n-------------') 
return pp_ids_data

NameError: name 'pvids1' is not defined

In [7]:
# In[4]:


date_in = '09/16/2022'
# date_out = date_in
date_out = '09/18/2022'

print(f'ERROR LOGS {date_in} to {date_out}\n')
cqr_results = getCQRResults(date_in,date_out)
cqr_inputs = getCQRInputs(cqr_results)
full_cqr_data = mergeCQRData(cqr_results, cqr_inputs)
pp_desc_data, pp_rules_data = getPPQAData(cqr_results['PVID'].unique().tolist())
#pp_ids_data=getPPIDData(cqr_results['PVID'].unique().tolist())
speed_audit_errors = generateSpeedAuditErrors(cqr_results, pp_desc_data)
flag_audit_errors = generateFlagAuditErrors(full_cqr_data, pp_rules_data)
df = completeErrorReport(speed_audit_errors,flag_audit_errors)
#df.to_csv('fullerrorreport.csv')


# In[6]: 




ERROR LOGS 09/16/2022 to 09/18/2022

Getting CQR data from Snowflake!
Success! Got CQR data from Snowflake. Number of rows: 18 
-------------
Getting CQR input data from S3!
Pulling from abandofanglers.com/abandofanglers.com_0913_02_27:02:27:49.csv
Pulling from www.loganhollowell.com/www.loganhollowell.com_0914_09_25:09:25:38.csv
Pulling from arcscissors.com/arcscissors.com_0914_20_19:08:19:11.csv
Pulling from prymal.com/prymal.com_0916_02_02:02:02:34.csv
Pulling from www.analogshift.com/partial_www.analogshift.com_0914_05_36:05:36:06.csv
Success! Got CQR input data from S3. Number of rows: 791 
-------------
Merging data!
Success! Merged data. Number of rows: 16 
-------------
Getting descriptions data from Snowflake!
Success! Got descriptions data from Snowflake. Number of rows: 43
Getting rules data from Snowflake!
Success! Got rules data from Snowflake. Number of rows: 12 
-------------
Generating Speed Audit errors!
Success! Generated Speed Audit Errors
-------------
Generating Fl

/var/folders/hn/mtn160v16kq8zlz1vr6zxzdh0000gn/T/ipykernel_39279/2688700813.py:214: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dff = df.groupby(tmp_cols)['VARIANT_PPED_AT','PVID','LINK'].min()                .reset_index()                .rename(columns = {'VARIANT_PPED_AT':'description_PPed_at','PVID':'sample_pvid','LINK':'sample_link'})
/var/folders/hn/mtn160v16kq8zlz1vr6zxzdh0000gn/T/ipykernel_39279/2688700813.py:243: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dff = df.groupby(cols)['VARIANT_PPED_AT','PVID','LINK'].min()            .reset_index()            .rename(columns = {'VARIANT_PPED_AT':'description_PPed_at','PVID':'sample_pvid','LINK':'sample_link'})


In [17]:
tdf = full_cqr_data.groupby(['CORRECT_DESCRIPTION'])['PVID'].nunique()
t2df = tdf.to_frame()
t2df

,PVID
CORRECT_DESCRIPTION,
"""There will be no more parades...""\n\nAfter the bells sounded on Armistice Day, the soldiers who had ""cursed through sludge"" in the trenches of the Great War brought home with them tools and technologies developed during the terrible conflict.\n\nThese were not just instruments of destruction, but life-changing--sometimes life-saving--innovations. Watches are no exception--in fact, it's in no small part due to the war that wristwatches became so popular.\n\nWhile soldiers had worn pocket watches strapped to the wrist since the 1860s, it wasn't until WWI that the need for precise timekeeping was recognized as paramount for strategy and tactical advantage. The introduction of aerial combat and timed artillery strikes necessitated a timepiece that could be read at a glance, and pocket watches simply would no longer suit that purpose. So soldiers strapped watches to their wrists, and carried them home at parade's end.\n\nCartier is one brand that popularized wristwatches ""over there."" On the home front, Hamilton and Elgin produced wristwatches for the American market.\n\nIngersoll, another American brand, got its start selling rubber stamps via mail order in the 1880s. By the 1890s they were selling watches produced by the Waterbury Clock Company. During the First World War, Ingersoll repurposed its Midget pocket watch for use on the wrists of American soldiers, starting a civilian trend once the War had ended.\n\nThe Radiolite wristwatch, introduced in 1919, incorporated another newfangled technology in its design: luminescence through Radium. Madame Curie's discovery was first used on watch dials produced by the U.S. Radium Corp in 1917, and Ingersoll started using radium on Radiolite pocket watches that same year.\n\nLuminescent dials proved invaluable in the low-light conditions of tanks and airplane cockpits, and after the War, watches with radium dials found use in civilian occupations such as motoring and camping. Just as Ingersoll became a watch word for value (the brand's Liberty watch, introduced in 1896, retailed for only one American dollar, which Ingersoll touted as ""the watch that made the dollar famous""), the Radiolite became the brand's most prolific model. ""\n\nThis expression bears a serial number dating it approximately to 1926, in the height of post-war wristwatch popularity. In keeping with its roots as a trench watch, the case is large even by today's standards, at 40mm. The stark black dial is highly legible, the puffy Arabic numerals prominent. Coming with its original strap and box (!!), the watch exudes a militant and sporty vibe, comfortably inhabiting the realm between military and civilian life, and is without question a brilliant piece of American watchmaking history.",1
"100% Handmade In Japan\n\nCreated for all-around haircutters seeking impeccable cutting performance, the PHANTOM II excels in wet, dry, precision and slide cutting while providing all-day comfort.\n\nFEATURES\n\nBest-selling scissor created for all cutting techniques\n\n100% handmade in Japan with Ultra Premium ATS314 steel\n\nUnique blade design won’t push or pull hair and achieve smooth, razor-sharp cutting\n\nErgonomic handle and permanent finger rest fits perfectly in the hand\n\nBall bearing pivot for smooth, controlled cutting\n\nNo snagging or pulling with flush-mounted tension adjuster\n\n_______________________________________________________________________________________________________\n\n""I love to use ARC Scissors when slice cutting around the face because they give a nice soft cut, great for adding subtle texture.""\n\n""I am absolutely obsessed with the 6"" PHANTOM II. They are the perfect shear for everything! The weightless feel along with a more narrow tip makes detail work effortless and precise.""\n\n""I love my 6” PHANTOM II from ARC™ Scissors because it is so versatile—it takes you through a haircut from wet to dry. They’re my go-to because they stay sharp and I love their

In [ ]:
df.to_clipboard(index = False)

In [ ]:
pp_ids_data

In [8]:
pp_desc_data

,USER_EMAIL,_ID,DESCRIPTION_ID,PVID,VARIANT_PPED_AT,AUDIT_LEVEL,QA_DESCRIPTION
0,joyce.palma@contractors.scale.com,62f903c6b3b189fb8c88c2d5,"[\n ""www.loganhollowell.com!10051814096932""\n]",www.loganhollowell.com!10051814096932,2022-08-14 14:16:38.143000+00:00,QA,SKU: LHR-1077-WGWD-6.0\n\n14k gold and diamond...
1,marco.escaroz@contractors.scale.com,631f7258eda8d68be515425a,"[\n ""www.analogshift.com!27705623553""\n]",www.analogshift.com!27705623553,2022-09-12 17:54:32.241000+00:00,QA,"""There will be no more parades...""\n\nAfter th..."
2,samantha.couoh@contractors.scale.com,631bc7517ba11e3dabf38234,"[\n ""abandofanglers.com!32121067700326"",\n ""...",abandofanglers.com!39748996431974,2022-09-09 23:08:01.653000+00:00,QA,
3,samantha.couoh@contractors.scale.com,631bc7517ba11e3dabf38234,"[\n ""abandofanglers.com!32121067700326"",\n ""...",abandofanglers.com!39620932141158,2022-09-09 23:08:01.653000+00:00,QA,
4,samantha.couoh@contractors.scale.com,631bc7517ba11ecf74f38232,"[\n ""abandofanglers.com!32121067700326"",\n ""...",abandofanglers.com!39748996431974,2022-09-09 23:08:01.634000+00:00,QA,"Minwaow 2.75""\n\nMinwaow 2.75"" Pro\n\nA Patric..."
5,samantha.couoh@contractors.scale.com,631bc7517ba11ecf74f38232,"[\n ""abandofanglers.com!32121067700326"",\n ""...",abandofanglers.com!39620932141158,2022-09-09 23:08:01.634000+00:00,QA,"Minwaow 2.75""\n\nMinwaow 2.75"" Pro\n\nA Patric..."
6,joyce.palma@contractors.scale.com,630d21e4f5575901ac61ef32,"[\n ""arcscissors.com!29528505483366"",\n ""arc...",arcscissors.com!29528505483366,2022-08-29 20:30:28.648000+00:00,QA,100% Handmade In Japan\nCreated for all-around...
7,joyce.palma@contractors.scale.com,630d21e4f5575901ac61ef32,"[\n ""arcscissors.com!29528505483366"",\n ""arc...",arcscissors.com!29528505286758,2022-08-29 20:30:28.648000+00:00,QA,100% Handmade In Japan\nCreated for all-around...
8,joyce.palma@contractors.scale.com,630d21e4f5575901ac61ef32,"[\n ""arcscissors.com!29528505483366"",\n ""arc...",arcscissors.com!29528505385062,2022-08-29 20:30:28.648000+00:00,QA,100% Handmade In Japan\nCreated for all-around...
9,marco.escaroz@contractors.scale.com,631fd44582b7c442948708ac,"[\n ""www.loganhollowell.com!10056636497956"",\...",www.loganhollowell.com!10056636170276,2022-09-13 00:52:21.854000+00:00,QA,"14k gold and diamond constellation ring, with ..."
